In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("python practice") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [19]:
spark
df=spark.read.format("csv")\
   .option("header","true")\
   .option("inferschema","true")\
   .load("./data/persons.csv")

df.show()
from pyspark.sql.functions import *
from pyspark.sql.window import Window




+------------------+---+------+------+
|              name|age|gender|salary|
+------------------+---+------+------+
|          John Doe| 30|  Male| 50000|
|        Jane Smith| 25|Female| 45000|
|     David Johnson| 35|  Male| 60000|
|       Emily Davis| 28|Female| 52000|
|    Michael Wilson| 40|  Male| 75000|
|       Sarah Brown| 32|Female| 58000|
|        Robert Lee| 29|  Male| 51000|
|       Lisa Garcia| 27|Female| 49000|
|    James Martinez| 38|  Male| 70000|
|Jennifer Rodriguez| 26|Female| 47000|
|  William Anderson| 33|  Male| 62000|
|   Karen Hernandez| 31|Female| 55000|
|Christopher Taylor| 37|  Male| 69000|
|     Mary Gonzalez| 24|Female| 44000|
|     Matthew Davis| 36|  Male| 67000|
|    Patricia White| 29|Female| 50000|
|     Daniel Miller| 34|  Male| 64000|
| Elizabeth Jackson| 30|Female| 52000|
|     Joseph Harris| 28|  Male| 53000|
|      Linda Martin| 39|Female| 71000|
+------------------+---+------+------+



In [3]:
df_flight=spark.read.format("csv")\
          .option("header","true")\
          .option("inferschema","true")\
          .load(r"C:\Users\venka\Downloads\2010-summary_1.csv")

df_station=spark.read.format("csv")\
          .option("header","true")\
          .option("inferschema","true")\
          .load(r"C:\Users\venka\Downloads\201508_station_data.csv")

In [ ]:
#df_flight.show()
#df_station.show()
window_spec = Window.orderBy(monotonically_increasing_id())
df_flight=df_flight.withColumn("station_id",row_number().over(window_spec))
#df_flight.show()

#df_joined_values=df_station.join(df_flight,on=["station_id"],how="inner")
#df_joined_values.show()
df_station.filter("station_id < 6") \
                  .join(df_flight, ( (df_station["station_id"]==df_flight["station_id"]) & (df_station["station_id"].isin(2,3,4))),"right_anti_join" )\
                  .select(df_flight["DEST_COUNTRY_NAME"]) \
                  .show()

#df_joined_values.show()

IllegalArgumentException: Unsupported join type 'right_anti_join'. Supported join types include: 'inner', 'outer', 'full', 'fullouter', 'full_outer', 'leftouter', 'left', 'left_outer', 'rightouter', 'right', 'right_outer', 'leftsemi', 'left_semi', 'semi', 'leftanti', 'left_anti', 'anti', 'cross'.

In [ ]:
from pyspark.sql.functions import col

df_station.filter("station_id < 6") \
    .join(df_flight, (df_station["station_id"] == df_flight["station_id"]) & df_station["station_id"].isin(2, 3, 4), "") \
    .select(col("DEST_COUNTRY_NAME")) \
    .show()


+--------------------+
|   DEST_COUNTRY_NAME|
+--------------------+
|       United States|
|       United States|
|       United States|
|               Egypt|
|                NULL|
|   Equatorial Guinea|
|       United States|
|       United States|
|          Costa Rica|
|             Senegal|
|       United States|
|              Guyana|
|       United States|
|               Malta|
|             Bolivia|
|            Anguilla|
|Turks and Caicos ...|
|       United States|
|Saint Vincent and...|
|               Italy|
+--------------------+
only showing top 20 rows



In [ ]:
#df_flight.count()
df_flight.filter(df_flight["DEST_COUNTRY_NAME"] == "Philippines" & ).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|      Philippines|      United States|  132|
+-----------------+-------------------+-----+



In [8]:
df_flight.groupBy("DEST_COUNTRY_NAME").max("count").withColumnRenamed("max(count)","max_of_values").show()

+--------------------+-------------+
|   DEST_COUNTRY_NAME|max_of_values|
+--------------------+-------------+
|            Anguilla|           21|
|              Russia|          152|
|            Paraguay|           90|
|             Senegal|           29|
|              Sweden|           65|
|            Kiribati|           17|
|              Guyana|           17|
|         Philippines|          132|
|            Malaysia|            1|
|           Singapore|           25|
|                Fiji|           53|
|              Turkey|           75|
|             Germany|         1392|
|         Afghanistan|           11|
|              Jordan|           50|
|               Palau|           31|
|Turks and Caicos ...|          136|
|              France|          774|
|              Greece|           50|
|              Taiwan|          275|
+--------------------+-------------+
only showing top 20 rows



In [12]:
df_flight.createOrReplaceTempView("flight_table")

In [16]:
spark.sql("select dest_country_name,sum(count) as sum_of_values from flight_table group by dest_country_name").show()

+--------------------+-------------+
|   dest_country_name|sum_of_values|
+--------------------+-------------+
|            Anguilla|           21|
|              Russia|          152|
|            Paraguay|           90|
|             Senegal|           29|
|              Sweden|           65|
|            Kiribati|           17|
|              Guyana|           17|
|         Philippines|          132|
|            Malaysia|            1|
|           Singapore|           25|
|                Fiji|           53|
|              Turkey|           75|
|             Germany|         1392|
|         Afghanistan|           11|
|              Jordan|           50|
|               Palau|           31|
|Turks and Caicos ...|          136|
|              France|          774|
|              Greece|           50|
|              Taiwan|          275|
+--------------------+-------------+
only showing top 20 rows



In [21]:
spark.conf.set("spark.sql.shuffle.partitions","5")

In [22]:
spark.stop()